In [ ]:
from fastmcp import FastMCP
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create FastMCP server
mcp = FastMCP("Weather Lookup Server")

@mcp.tool()
def get_weather_by_zipcode(zipcode: str) -> dict:
    """
    Get current weather for a US zip code from WeatherAPI.com
    
    Args:
        zipcode: US zip code (e.g., "10001")
    
    Returns:
        Dictionary containing weather information
    """
    try:
        # Get API key from environment
        api_key = os.getenv('WEATHER_API_KEY')
        
        if not api_key:
            # Free tier: Get key from https://www.weatherapi.com/signup.aspx
            return {
                "error": "WEATHER_API_KEY not found in environment variables",
                "message": "Get a free API key from https://www.weatherapi.com/signup.aspx"
            }
        
        # WeatherAPI.com endpoint
        url = f"http://api.weatherapi.com/v1/current.json"
        params = {
            "key": api_key,
            "q": zipcode,
            "aqi": "no"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        # Format the response
        weather_info = {
            "location": f"{data['location']['name']}, {data['location']['region']}",
            "zipcode": zipcode,
            "temperature_f": data['current']['temp_f'],
            "temperature_c": data['current']['temp_c'],
            "condition": data['current']['condition']['text'],
            "humidity": data['current']['humidity'],
            "wind_mph": data['current']['wind_mph'],
            "feels_like_f": data['current']['feelslike_f'],
            "last_updated": data['current']['last_updated']
        }
        
        return weather_info
        
    except requests.exceptions.RequestException as e:
        return {
            "error": "Failed to fetch weather data",
            "details": str(e)
        }
    except KeyError as e:
        return {
            "error": "Failed to parse weather data",
            "details": f"Missing key: {str(e)}"
        }

@mcp.tool()
def get_weather_summary(zipcode: str) -> str:
    """
    Get a human-readable weather summary for a zip code
    
    Args:
        zipcode: US zip code (e.g., "10001")
    
    Returns:
        String with weather summary
    """
    weather = get_weather_by_zipcode(zipcode)
    
    if "error" in weather:
        return f"Error: {weather['error']}"
    
    return f"""Weather for {weather['location']} (Zip: {weather['zipcode']}):
🌡️ Temperature: {weather['temperature_f']}°F ({weather['temperature_c']}°C)
🌤️ Conditions: {weather['condition']}
🤚 Feels like: {weather['feels_like_f']}°F
💧 Humidity: {weather['humidity']}%
💨 Wind: {weather['wind_mph']} mph
⏰ Last updated: {weather['last_updated']}"""

print("✓ FastMCP Weather Server configured")
print("Available tools:")
print("  - get_weather_by_zipcode(zipcode)")
print("  - get_weather_summary(zipcode)")


In [ ]:
# Test the weather lookup (uncomment to run)
# Call through the MCP tool registry
result = mcp.tools["get_weather_by_zipcode"].fn("10001")

In [ ]:
# Test the weather summary (uncomment to run)
# summary = get_weather_summary("90210")  # Beverly Hills
# print(summary)


In [ ]:
# Run the MCP server
print("Starting Weather MCP Server...")
mcp.run()
